In [1]:
import contextlib
from functools import cached_property
from pathlib import Path

import h5py

In [2]:
pathobj = Path("amke_testfile.h5").resolve()
with h5py.File(pathobj) as h5f:
    print(h5f.keys())

<KeysViewHDF5 ['Sums', 'UserDataCfg', 'c_piranha', 'crix_w8', 'det_crix_w8', 'det_rix_fim0', 'det_rix_fim1', 'epics_archiver', 'intg', 'lightStatus', 'mono_hrencoder', 'rix_fim0', 'rix_fim1', 'scan', 'timestamp', 'timing']>


In [ ]:
class Detector():

    def __init__(self, group: h5py.Group, data_to_read: dict):
        # group = first level group, e.g. andordir, data_to_read: lower level data in andor_dir
        self.__grp = group
        self.prop_factory(data_to_read)

    def prop_set(self, data_set):
        def fget(self):
            return self.__grp[data_set][()]
        return fget


    def prop_factory(self, data_to_read: dict):
        """Set up all the desired cached properties."""
        for name, dataset in data_to_read.items():
            prop=cached_property(self.prop_set(dataset))
            setattr(self.__class__, name,prop)
            prop.__set_name__(self.__class__, name)

andor_dir_dict = {
    'count': 'count',
    'full_area': 'full_area',
    'eventcodes': 'timing_sum_eventcodes',
    'apds': 'det_crix_w8_sum_full_area',
    'fim_0': 'det_rix_fim0_sum_full_area',
    'fim_1': 'det_rix_fim1_sum_full_area',
    'mono_encoder': 'mono_hrencoder_sum_value',
    'piranha': 'c_piranha_sum_full_area'}
andor_vls_dict = andor_dir_dict # assuming both detectors have the same keys

axis_svls_dict = {
    'count': 'count',
    'full_area': 'full_area',
}

detectors = {
    'andor_dir': {'attrdict': andor_dir_dict, 'clsname': 'AndorDir'},
    'andor_vls': {'attrdict': andor_vls_dict, 'clsname': 'AndorVLS'},
    'axis_svls': {'attrdict': axis_svls_dict, 'clsname': 'AxisSVLS'},
}


class Integrating():

    """
    Class for accessing all data related to 
    """

    def __init__(self, intgrp: h5py.Group):
        for detector in detectors:
            if detector in intgrp.keys():
                detobj = type(detectors[detector]["clsname"], (Detector,), {})
                setattr(self, detector, detobj(intgrp[detector], detectors[detector]['attrdict']))

    def __getattr__(self, name):
        if name in detectors:
            raise KeyError(f"{name} not in this file.")
        return super().__getattribute__(name)

class Singleshot():

    def __init__(self, ssgrp: h5py.Group):
        grp = ssgrp 
        Piranha = type("Piranha", (Detector,), {})
        self.piranha = Piranha(grp, {"piranha": "c_piranha"})

class SmallData():
    """
    A  class for fast read-only interface to our small data files.

    Parameters
    ----------
    path : str or Path or h5py File or Group object
        The filename to read from.
    """
    
    def __init__(self, path: str | Path | h5py.File | h5py.Group):
        self.__file = None
        if isinstance(path, h5py.File):
            self.path = Path(path.filename).resolve()
            self.__file = path
        elif isinstance(path, h5py.Group):
            self.path = Path(path.file.filename).resolve()
            self.__file = path.file
        elif isinstance(path, str | Path):
            self.path = Path(path).resolve()
     
    def is_open(self) -> bool:
        """Whether the file is open."""
        return bool(self.__file)

    def __del__(self):
        """Close the file when the object is deleted."""
        if self.__file:
            self.__file.close()

    def close(self):
        """Close the file."""
        self.integrating = None

        with contextlib.suppress(AttributeError):
            del self.__intgrp

        if self.__file:
            self.__file.close()
        self.__file = None

    def open(self):  # noqa: A003
        """Open the file."""
        if not self.__file:
            self.__file = h5py.File(self.path, "r")
            self.__intgrp = self.__file["/intg"]
            self.__ssgrp = self.__file["/"]

    @cached_property
    def integrating(self) -> h5py.Group:
        """Get the integrating object."""
        if not self.__file:
            self.open()
        return Integrating(self.__intgrp)

    @cached_property
    def single_shot(self) -> h5py.Group:
        """Get the integrating object."""
        if not self.__file:
            self.open()
        return Singleshot(self.__ssgrp)


In [4]:
smdata = SmallData(pathobj)

In [9]:
andor = smdata.integrating.andor_dir

In [10]:
andor.apds

array([0], dtype=int32)

In [ ]:
smdata.single_shot.piranha